In [1]:
%pip install ollama langchain langchain_community langchain_core pypdf chromadb

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.18.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.20.1-cp313-cp313-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.30.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_instrume

In [2]:
import ollama
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings

In [ ]:
import chromadb
print(chromadb.__version__)

0.6.3


In [3]:
loader = PyPDFLoader("./catalogo_.pdf")
pages = loader.load()
# 
len(pages)

18

In [4]:
print(pages[1].page_content)

VENTA DE EQUIPOS DE COMPUTO - LAPTOPS 
 
 
ANTES: S/. 800 
 
DELL LATITUDE 3350 S/. 650 
 
LAPTOP DELL LATITUDE 3350 / LINEA EMPRE- 
SARIAL DE ALTA CALIDAD / Procesador Intel 
Core i3-5005u 2.0 Ghz 3mb Cache / Memoria 
RAM 8GB / Disco Solido 128GB SSD / Pantalla 
Led 13.3" Panoramica / Resolucion 1366 x 768p 
(HD) / Incluye WebCam + Microfono / Video 
Integrado Intel HD 5500 / TECLADO EN ESPA- 
ÑOL / 01 Wi-Fi 802.11a/b/g/n/ac / 01, Bluetoo- 
th 4.0 / 01 Mini DisplayPort / 01 HDMI / 02 USB 
3.1 Gen 1 / 01 USB (Power Share) / 01 LAN RJ45 
100/1000 / 01 Lector SD Memory Card / 01 
Conector Audifono y Microfono / Bateria 6 
Celdas Litio 65wh / Cargador AC 110 / 230v/ 
GARANTIA 01 AÑO / Producto REFURBISHED / 
RENOVADAS AL 100% EN CAJA COMO NUEVAS 
A1 / PRECIO REMATE S/. 650 
CCOOMMOO 
NNUUEEVVOO 
SS//..665500 
 
 
 
13.3” 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
LENOVO THINKPAD T470 S/. 750 
 
Laptop Lenovo Thinkpad T470 / Procesador 
Intel Core i5-7200u 2.5Ghz / Max. Turbo 


In [5]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=250,
    chunk_overlap=80,
    length_function=len
)

docs = text_splitter.split_documents(pages)

In [6]:
len(docs)

127

In [7]:
llm = "deepseek-r1:1.5b"

In [ ]:
lm_embedder = "nomic-embed-text"

In [11]:
# Inicializa el modelo de embeddings de Ollama
embeddings = OllamaEmbeddings(model=lm_embedder)

vectorstore = Chroma.from_documents(documents= docs, embedding= embeddings, persist_directory="./conocimiento_catalogo")

In [ ]:
def ollama_llm(question, context):
    system_prompt = """Eres un asistente virtual especializado en la venta de productos. Tu objetivo es ayudar a los clientes de manera amable y cordial, asegurando que comprendan bien las opciones disponibles. Sé educado, usa un tono amigable y brinda sugerencias claras y útiles.

Cuando no sepas la respuesta, responde con empatía y ofrece alternativas. Ejemplo:
- "Lo siento, pero no tengo información sobre eso en este momento. ¿Te gustaría que te ayude con otra consulta?"

Usa frases como:
- "¡Por supuesto! Aquí tienes algunas opciones..."
- "Me encantaría ayudarte con eso..."
- "¡Qué buena pregunta! Déjame mostrarte las mejores opciones...

Responde siempre en español"
"""

    formatted_prompt = f"Pregunta: {question} \n\n Contexto (responde solo sobre el contenido del contexto entregado): {context}"

    response = ollama.chat(
        model=llm,
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': formatted_prompt}
        ],
        options={'temperature': 0.1}
    )

    return response['message']['content']


In [13]:
retriever = vectorstore.as_retriever()
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
def rag_chain(question,):
    retrieved_docs = retriever.invoke(question,)
    formated_context = combine_docs(retrieved_docs)
    return ollama_llm(question, formated_context)

In [18]:
def menu_interactivo():
    print("\n✨ ¡Bienvenido al Asistente de Ventas! ✨")
    print("🛍️ Estoy aquí para ayudarte a encontrar los mejores productos.")
    print("🤖 Pregunta lo que necesites o escribe 'salir' para terminar.\n")

    while True:
        pregunta = input("💬 Tú: ")

        if pregunta.lower() == "salir":
            print("\n🤖 Asistente: ¡Gracias por tu consulta! 😊 Que tengas un día increíble. 🌟")
            break

        retrieved_docs = retriever.invoke(pregunta)
        formatted_context = combine_docs(retrieved_docs)
        respuesta = ollama_llm(pregunta, formatted_context)

        print(f"\n🤖 Asistente: {respuesta}\n")

# Ejecutar el menú
menu_interactivo()



✨ ¡Bienvenido al Asistente de Ventas! ✨
🛍️ Estoy aquí para ayudarte a encontrar los mejores productos.
🤖 Pregunta lo que necesites o escribe 'salir' para terminar.


🤖 Asistente: <think>
Primero, el cliente está pidiendo un laptop para realizar programación. Necesito determinar si los recommendados cumplen con esta necesidad.

Voy a analizar los modelos disponibles:

1. **DELL Latitude 7400**: Es una computadora portála con procesor Intel Core i7-8665U, 1.9 Ghz y 8 MB de cache. Tiene discounto funcional 100% y prezzo offsite 950.

2. **HP ProBOOK 640 G5**: Proveedor de computadora portála con procesor Intel Core i5-8365U, 1.6 Ghz y 2 núcleos. Tiene discounto funcional 100% y prezzo offsite 950.

3. **DELL Latitude 5400**: Proveedor de computadora portála con procesor Intel Core i5-8365U, 1.6 Ghz y 4 núcleos. Tiene discounto funcional 100% y prezzo offsite 900.

Los modelos incluyen procesores de CPU básicos, lo que podría ser suficiente para programación básica. Sin embargo, los proce